# W&B Example - LLM
In this tutorial we will walk through a practical use case of using the Giskard LLM Scan and the W&B tracer on a Prompt Chaining task, one step at a time.

### Prerequisites 🔧
To begin, it is essential to:

- have a Python version between 3.8 and 3.11 and the following PyPI packages:
    - `wandb` (for more installation instructions, read this [page](https://docs.wandb.ai/guides/hosting/how-to-guides/basic-setup))
    - `giskard[llm]` (for more installation instructions, read this [page](https://docs.giskard.ai/en/latest/guides/installation_library/index.html))
    
    
- sign up for a Weights and Biases account [here](https://wandb.ai/site).

In [ ]:
!pip install wandb giskard[llm] -q

In [ ]:
import wandb
wandb.login(key="key to retrieve from https://wandb.ai/authorize")

### Configurations 🔩

Next, let's configure 3 environment variables:

- `OPENAI_API_KEY`: where you would provide your own OpenAI ChatGPT API key (More instructions [here](https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key)).
- `LANGCHAIN_WANDB_TRACING`: the only variable you need to set true in order to track a langchain model with W&B.
- `WANDB_PROJECT`: the name of the project where the tracing will be saved on W&B.

In [ ]:
import os

# Setting up OpenAI API KEY
os.environ['OPENAI_API_KEY'] = "sk-xxx"

# Enabling the W&B tracing
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"

# Picking up a name for the project
os.environ["WANDB_PROJECT"] = "product_description"

### Introduction to the use-case ✍️

Many of the World’s most successful companies (see this [blog](https://concured.com/blog/5-brands-that-are-successfully-leveraging-ai-for-marketing/)) are leveraging machine learning and AI technology as an integral part of their marketing strategy.

In the world of advertising, efficient product marketing goes beyond just grabbing attention. It's about offering comprehensive product descriptions to enhance visibility, attract quality leads, and build a strong brand image. Yet, manually crafting these descriptions can be time-consuming and repetitive.

In the following, we will show with a basic example how this process can be simplified. Given a product name, we will ask the LLM to process 2 chained prompts using `langchain` in order to provide us with a product description. The 2 prompts can be described as follows:

1. `keywords_prompt_template`: Based on the product name (given by the user), the LLM has to provide a list of five to ten relevant keywords that would increase product visibility.
2. `product_prompt_template`: Based on the given keywords (given as a response to the first prompt), the LLM has to generate a multi-paragraph rich text product description with emojis that is creative and SEO compliant.

In [ ]:
from langchain.prompts import ChatPromptTemplate

# First prompt to generate keywords related to the product name
keywords_prompt_template = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant that generate a CSV list of keywords related to a product name

    Example Format:
    PRODUCT NAME: product name here
    KEYWORDS: keywords separated by commas here

    Generate five to ten keywords that would increase product visibility. Begin!

    """),
    ("human", """
    PRODUCT NAME: {product_name}
    KEYWORDS:""")])

# Second chained prompt to generate a description based on the given keywords from the first prompt
product_prompt_template = ChatPromptTemplate.from_messages([
    ("system", """As a Product Description Generator, generate a multi paragraph rich text product description with emojis based on the information provided in the product name and keywords separated by commas.

    Example Format:
    PRODUCT NAME: product name here
    KEYWORDS: keywords separated by commas here
    PRODUCT DESCRIPTION: product description here

    Generate a product description that is creative and SEO compliant. Emojis should be added to make product description look appealing. Begin!

    """),
    ("human", """
    PRODUCT NAME: {product_name}
    KEYWORDS: {keywords}
    PRODUCT DESCRIPTION:
        """)])

### Initialization of the LLMs ⛓️

We can now create the two `langchain` models powered by `gpt-3.5-turbo` and `gpt-4` . In order to facilitate the organization and retrieval of the different models and results, we will create a small dictionary that will contain for each foundational model:

- `langchain`: the langchain model.
- `giskard`: the giskard wrapper that will eventually be used by the scan.
- `scan_report`: the report resulting from running the scan.
- `test_suite`: the test suite and metrics generated by the scan.

In [ ]:
models = {"gpt-3.5-turbo": {"langchain": None, "giskard": None, "scan_report": None, "test_suite": None},
          "gpt-4": {"langchain": None, "giskard": None, "scan_report": None, "test_suite": None},}

Using the prompt templates defined earlier we can create two LLMChain and concatenate them into a SequentialChain that takes as input the product name, and outputs a product description

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

for model in models.keys():

    # langchain model powered by ChatGPT
    llm = ChatOpenAI(temperature=0.2, model=model)

    # Defining the chains
    keywords_chain = LLMChain(llm=llm, prompt=keywords_prompt_template, output_key="keywords")
    product_chain = LLMChain(llm=llm, prompt=product_prompt_template, output_key="description")

    # Concatenation of both chains
    models[model]["langchain"] = SequentialChain(chains=[keywords_chain, product_chain], input_variables=["product_name"],
                                    output_variables=["description"])

### Wrapping of the LLMs with Giskard 🎁

In order to perform the scan, we will wrap the previously defined langchain models with `giskard.Model` API that takes 4 important arguments:

- `model`: the model that we would like to wrap, in this case `models[model]["langchain"]` defined above.
- `model_type`: the type of the model in question in order to communicate it to giskard.
- `description`: the description of the model’s task. This is very important as it will be used to generate internal prompts and evaluation strategies to scan the model.
- `feature_names`: the name of the model’s input.

In [ ]:
import giskard

for model in models.keys():
    models[model]["giskard"] = giskard.Model(models[model]["langchain"], name="Product keywords and description generator", model_type="text_generation",
                          description="Generate product description based on a product's name and the associated keywords. "
                                      "Description should be using emojis and being SEO compliant.",
                          feature_names=['product_name'])

We will also wrap a small dataset that will be used during the scan. This is an optional step, as in the absence of a dataset, the scan will automatically generate a representative one based on the description provided by the `giskard.Model` API.

In [ ]:
import pandas as pd
pd.set_option("display.max_colwidth", 999)

dataset = giskard.Dataset(pd.DataFrame({
    'product_name': ["Double-Sided Cooking Pan",
                     "Automatic Plant Watering System",
                     "Miniature Exercise Equipment"],

}), name="Test dataset",
    column_types={"product_name": "text"})

### Examples

In [ ]:
import wandb
run = wandb.init(project=os.environ["WANDB_PROJECT"], name="examples")
predictions = models["gpt-3.5-turbo"]["giskard"].predict(dataset).prediction
for k, v in dataset.df.product_name.to_dict().items():
  os.environ["WANDB_NAME"] = "examples_"+str(k)
  print("Example #", k+1)
  print("product_name (input):", v)
  print("product_description (output):", predictions[k])
  print("--------------------------------------------------------------------")
run.finish()

### Scanning with Giskard and Logging into W&B 🔬

At this point, we have all the ingredients we need to perform the scan on the two models. Not only we will be finding issues automatically, we will be able to have a full trace of every prompt and response used to find them!

To run the scan, we will run over the two models, initiate a new W&B run inside our created project (in order to separate the traces), run the one-liner `giskard.scan` API on the wrapped model and dataset, generate a test suite and finally log the results into W&B.

In [ ]:
for model in models.keys():
	# Initiate a new run with the foundational model name inside the W&B project
    run = wandb.init(project=os.environ["WANDB_PROJECT"], name=model)

    # Scan report
	# 1) Generate
    models[model]['scan_report'] = giskard.scan(models[model]['giskard'], dataset, raise_exceptions=True)
    # 2) Log into W&B
    models[model]['scan_report'].to_wandb(run)

    # Test suite
	# 1) Generate
    models[model]['test_suite'] = models[model]['scan_report'].generate_test_suite()
    # 2) Log into W&B
    models[model]['test_suite'].run().to_wandb(run)

	# End W&B run
    run.finish()

### Scan results 🎉

In order to visualise the scan reports, you can either navigate to your https://wandb.ai/home or if you’re running in a notebook all you have to do is execute the following line in a cell:

In [21]:
display(models["gpt-3.5-turbo"]['scan_report'])

<iframe id="scan-137977027586192" srcdoc="<!doctype html>
<html lang="en">

<head>
 <title>Giskard Scan Results</title>
 <base target="_blank">
 <meta charset="utf-8">
 
<style>pre code.hljs{display:block;overflow-x:auto;padding:1em}code.hljs{padding:3px 5px}
/*!
 Theme: GitHub Dark
 Description: Dark theme as seen on github.com
 Author: github.com
 Maintainer: @Hirse
 Updated: 2021-05-15

 Outdated base version: https://github.com/primer/github-syntax-dark
 Current colors taken from GitHub's CSS
*/.hljs{background:#0d1117;color:#c9d1d9}.hljs-doctag,.hljs-keyword,.hljs-meta .hljs-keyword,.hljs-template-tag,.hljs-template-variable,.hljs-type,.hljs-variable.language_{color:#ff7b72}.hljs-title,.hljs-title.class_,.hljs-title.class_.inherited__,.hljs-title.function_{color:#d2a8ff}.hljs-attr,.hljs-attribute,.hljs-literal,.hljs-meta,.hljs-number,.hljs-operator,.hljs-selector-attr,.hljs-selector-class,.hljs-selector-id,.hljs-variable{color:#79c0ff}.hljs-meta .hljs-string,.hljs-regexp,.hljs-string{color:#a5d6ff}.hljs-built_in,.hljs-symbol{color:#ffa657}.hljs-code,.hljs-comment,.hljs-formula{color:#8b949e}.hljs-name,.hljs-quote,.hljs-selector-pseudo,.hljs-selector-tag{color:#7ee787}.hljs-subst{color:#c9d1d9}.hljs-section{color:#1f6feb;font-weight:700}.hljs-bullet{color:#f2cc60}.hljs-emphasis{color:#c9d1d9;font-style:italic}.hljs-strong{color:#c9d1d9;font-weight:700}.hljs-addition{background-color:#033a16;color:#aff5b4}.hljs-deletion{background-color:#67060c;color:#ffdcd7}.hljs-copy-wrapper{overflow:hidden;position:relative}.hljs-copy-button:focus,.hljs-copy-wrapper:hover .hljs-copy-button{transform:translateX(0)}.hljs-copy-button{background-color:#2d2b57;background-color:var(--hljs-theme-background);background-image:url('data:image/svg+xml;charset=utf-8,<svg xmlns="http://www.w3.org/2000/svg" width="16" height="16" fill="none" viewBox="0 0 24 24"><path fill="%23fff" fill-rule="evenodd" d="M6 5a1 1 0 0 0-1 1v14a1 1 0 0 0 1 1h12a1 1 0 0 0 1-1V6a1 1 0 0 0-1-1h-2a1 1 0 1 1 0-2h2a3 3 0 0 1 3 3v14a3 3 0 0 1-3 3H6a3 3 0 0 1-3-3V6a3 3 0 0 1 3-3h2a1 1 0 0 1 0 2H6Z" clip-rule="evenodd"/><path fill="%23fff" fill-rule="evenodd" d="M7 3a2 2 0 0 1 2-2h6a2 2 0 0 1 2 2v2a2 2 0 0 1-2 2H9a2 2 0 0 1-2-2V3Zm8 0H9v2h6V3Z" clip-rule="evenodd"/></svg>');background-position:50%;background-repeat:no-repeat;border:1px solid #ffffff22;border-radius:.25rem;color:#fff;height:2rem;position:absolute;right:1em;text-indent:-9999px;top:1em;transition:background-color .2s ease,transform .2s ease-out;width:2rem}.hljs-copy-button:hover{border-color:#ffffff44}.hljs-copy-button:active{border-color:#ffffff66}.hljs-copy-button[data-copied=true]{background-image:none;text-indent:0;width:auto}@media (prefers-reduced-motion){.hljs-copy-button{transition:none}}.hljs-copy-alert{clip:rect(0 0 0 0);-webkit-clip-path:inset(50%);clip-path:inset(50%);height:1px;overflow:hidden;position:absolute;white-space:nowrap;width:1px}
/*! tailwindcss v3.3.2 | MIT License | https://tailwindcss.com*/*,:after,:before{border:0 solid #e5e7eb;box-sizing:border-box}:after,:before{--tw-content:""}html{-webkit-text-size-adjust:100%;font-feature-settings:normal;font-family:ui-sans-serif,system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji;font-variation-settings:normal;line-height:1.5;-moz-tab-size:4;-o-tab-size:4;tab-size:4}body{line-height:inherit;margin:0}hr{border-top-width:1px;color:inherit;height:0}abbr:where([title]){-webkit-text-decoration:underline dotted;text-decoration:underline dotted}h1,h2,h3,h4,h5,h6{font-size:inherit;font-weight:inherit}a{color:inherit;text-decoration:inherit}b,strong{font-weight:bolder}code,kbd,pre,samp{font-family:ui-monospace,SFMono-Regular,Menlo,Monaco,Consolas,Liberation Mono,Courier New,monospace;font-size:1em}small{font-size:80%}sub,sup{font-size:75%;line-height:0;position:relative;vertical-align:baseline}sub{bottom:-.25em}sup{top:-.5em}table{border-collaps

In [22]:
display(models["gpt-4"]['scan_report'])

<iframe id="scan-137977027888128" srcdoc="<!doctype html>
<html lang="en">

<head>
 <title>Giskard Scan Results</title>
 <base target="_blank">
 <meta charset="utf-8">
 
<style>pre code.hljs{display:block;overflow-x:auto;padding:1em}code.hljs{padding:3px 5px}
/*!
 Theme: GitHub Dark
 Description: Dark theme as seen on github.com
 Author: github.com
 Maintainer: @Hirse
 Updated: 2021-05-15

 Outdated base version: https://github.com/primer/github-syntax-dark
 Current colors taken from GitHub's CSS
*/.hljs{background:#0d1117;color:#c9d1d9}.hljs-doctag,.hljs-keyword,.hljs-meta .hljs-keyword,.hljs-template-tag,.hljs-template-variable,.hljs-type,.hljs-variable.language_{color:#ff7b72}.hljs-title,.hljs-title.class_,.hljs-title.class_.inherited__,.hljs-title.function_{color:#d2a8ff}.hljs-attr,.hljs-attribute,.hljs-literal,.hljs-meta,.hljs-number,.hljs-operator,.hljs-selector-attr,.hljs-selector-class,.hljs-selector-id,.hljs-variable{color:#79c0ff}.hljs-meta .hljs-string,.hljs-regexp,.hljs-string{color:#a5d6ff}.hljs-built_in,.hljs-symbol{color:#ffa657}.hljs-code,.hljs-comment,.hljs-formula{color:#8b949e}.hljs-name,.hljs-quote,.hljs-selector-pseudo,.hljs-selector-tag{color:#7ee787}.hljs-subst{color:#c9d1d9}.hljs-section{color:#1f6feb;font-weight:700}.hljs-bullet{color:#f2cc60}.hljs-emphasis{color:#c9d1d9;font-style:italic}.hljs-strong{color:#c9d1d9;font-weight:700}.hljs-addition{background-color:#033a16;color:#aff5b4}.hljs-deletion{background-color:#67060c;color:#ffdcd7}.hljs-copy-wrapper{overflow:hidden;position:relative}.hljs-copy-button:focus,.hljs-copy-wrapper:hover .hljs-copy-button{transform:translateX(0)}.hljs-copy-button{background-color:#2d2b57;background-color:var(--hljs-theme-background);background-image:url('data:image/svg+xml;charset=utf-8,<svg xmlns="http://www.w3.org/2000/svg" width="16" height="16" fill="none" viewBox="0 0 24 24"><path fill="%23fff" fill-rule="evenodd" d="M6 5a1 1 0 0 0-1 1v14a1 1 0 0 0 1 1h12a1 1 0 0 0 1-1V6a1 1 0 0 0-1-1h-2a1 1 0 1 1 0-2h2a3 3 0 0 1 3 3v14a3 3 0 0 1-3 3H6a3 3 0 0 1-3-3V6a3 3 0 0 1 3-3h2a1 1 0 0 1 0 2H6Z" clip-rule="evenodd"/><path fill="%23fff" fill-rule="evenodd" d="M7 3a2 2 0 0 1 2-2h6a2 2 0 0 1 2 2v2a2 2 0 0 1-2 2H9a2 2 0 0 1-2-2V3Zm8 0H9v2h6V3Z" clip-rule="evenodd"/></svg>');background-position:50%;background-repeat:no-repeat;border:1px solid #ffffff22;border-radius:.25rem;color:#fff;height:2rem;position:absolute;right:1em;text-indent:-9999px;top:1em;transition:background-color .2s ease,transform .2s ease-out;width:2rem}.hljs-copy-button:hover{border-color:#ffffff44}.hljs-copy-button:active{border-color:#ffffff66}.hljs-copy-button[data-copied=true]{background-image:none;text-indent:0;width:auto}@media (prefers-reduced-motion){.hljs-copy-button{transition:none}}.hljs-copy-alert{clip:rect(0 0 0 0);-webkit-clip-path:inset(50%);clip-path:inset(50%);height:1px;overflow:hidden;position:absolute;white-space:nowrap;width:1px}
/*! tailwindcss v3.3.2 | MIT License | https://tailwindcss.com*/*,:after,:before{border:0 solid #e5e7eb;box-sizing:border-box}:after,:before{--tw-content:""}html{-webkit-text-size-adjust:100%;font-feature-settings:normal;font-family:ui-sans-serif,system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji;font-variation-settings:normal;line-height:1.5;-moz-tab-size:4;-o-tab-size:4;tab-size:4}body{line-height:inherit;margin:0}hr{border-top-width:1px;color:inherit;height:0}abbr:where([title]){-webkit-text-decoration:underline dotted;text-decoration:underline dotted}h1,h2,h3,h4,h5,h6{font-size:inherit;font-weight:inherit}a{color:inherit;text-decoration:inherit}b,strong{font-weight:bolder}code,kbd,pre,samp{font-family:ui-monospace,SFMono-Regular,Menlo,Monaco,Consolas,Liberation Mono,Courier New,monospace;font-size:1em}small{font-size:80%}sub,sup{font-size:75%;line-height:0;position:relative;vertical-align:baseline}sub{bottom:-.25em}sup{top:-.5em}table{border-collaps